In [1]:
import os
import zipfile
import numpy as np
from tkinter import Tk, filedialog
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
# --- Data Extraction Section ---
def extract_dataset(dataset_path, extracted_path):
    """Extracts the dataset from a zip file."""
    if not os.path.exists(extracted_path):
        try:
            with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_path)
            print("Dataset extracted successfully.")
        except FileNotFoundError:
            print(f"Error: Zip file not found at {dataset_path}")
        except zipfile.BadZipFile:
            print(f"Error: Invalid zip file at {dataset_path}")
        except Exception as e:
            print(f"An error occurred during extraction: {e}")
    else:
        print("Dataset already extracted.")

In [3]:
# --- Main Program Flow ---
Tk().withdraw()  # Hide the root Tkinter window
print("Please select the 'plantdataset.zip' file.")
dataset_path = filedialog.askopenfilename(title="Select Plant Dataset Zip File", filetypes=[("Zip files", "*.zip")])

if not dataset_path:
    print("Error: No file selected. Exiting.")
    exit()

extracted_path = os.path.join(os.path.dirname(dataset_path), "plantdataset")
extract_dataset(dataset_path, extracted_path)


Please select the 'plantdataset.zip' file.
Dataset already extracted.


In [4]:
# --- Data Preparation and Model Building ---
plantvillage_path = os.path.join(extracted_path, "PlantVillage")

if not os.path.exists(plantvillage_path):
    print("Error: 'PlantVillage' folder not found. Check dataset structure. Exiting.")
    exit()

train_dir = os.path.join(plantvillage_path, "train")
val_dir = os.path.join(plantvillage_path, "val")

if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    print("Error: Training/validation dirs missing. Check structure. Exiting.")
    exit()

In [5]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 43444 images belonging to 38 classes.
Found 10861 images belonging to 38 classes.


In [6]:
# Build the ANN model
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

C:\Users\Ayaan\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # You can adjust the number of epochs
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/10


C:\Users\Ayaan\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1357/1357 ━━━━━━━━━━━━━━━━━━━━ 914s 665ms/step - accuracy: 0.1632 - loss: 3.2150 - val_accuracy: 0.3435 - val_loss: 2.3167
Epoch 2/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 130us/step - accuracy: 0.3750 - loss: 2.2648 - val_accuracy: 0.3077 - val_loss: 2.1439
Epoch 3/10


F:\LDPlayer\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1357/1357 ━━━━━━━━━━━━━━━━━━━━ 287s 211ms/step - accuracy: 0.3163 - loss: 2.4114 - val_accuracy: 0.3866 - val_loss: 2.1122
Epoch 4/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 107us/step - accuracy: 0.3438 - loss: 2.3461 - val_accuracy: 0.2308 - val_loss: 2.6078
Epoch 5/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 283s 208ms/step - accuracy: 0.3492 - loss: 2.2319 - val_accuracy: 0.4095 - val_loss: 1.9749
Epoch 6/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 53us/step - accuracy: 0.3750 - loss: 2.2890 - val_accuracy: 0.5385 - val_loss: 1.7481
Epoch 7/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 278s 204ms/step - accuracy: 0.3783 - loss: 2.1065 - val_accuracy: 0.4522 - val_loss: 1.8894
Epoch 8/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 77us/step - accuracy: 0.4688 - loss: 2.0088 - val_accuracy: 0.4615 - val_loss: 2.1247
Epoch 9/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 283s 208ms/step - accuracy: 0.4123 - loss: 1.9789 - val_accuracy: 0.4725 - val_loss: 1.7609
Epoch 10/10
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 77us/step - accuracy: 0.5312 - loss

In [9]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


340/340 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.4754 - loss: 1.7567
Validation Accuracy: 47.63%


In [10]:
# Save the trained model
model_save_path = os.path.join(os.path.dirname(dataset_path), "plant_disease_ann_model.h5") # Fixed path
model.save(model_save_path)
print(f"Model saved as '{os.path.basename(model_save_path)}'.")

Model saved as 'plant_disease_ann_model.h5'.
